---
title: Models
format:
  html:
    code-fold: true
---

Need to run command in shell first as `pipeline` is project-specific command

```{sh}
kedro pipeline create model
```

In [ ]:
#| default_exp pipelines/model/pipeline

In [ ]:
#| hide
%load_ext lineapy
%load_ext autoreload
%autoreload 2

In [ ]:
import hvplot.pandas
import lineapy
import seaborn as sns

In [ ]:
#| export
from loguru import logger
from typing import Union, Collection, Callable, Optional, Tuple
from typing import Any, Dict


import numpy as np
import polars as pl
import pandas

In [ ]:
from ids_finder.utils.basic import pl_norm, resample

#### `Kerdo`

In [ ]:
#| export
from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline


In [ ]:
from ids_finder.utils.basic import load_catalog

In [ ]:
catalog = load_catalog()

In [ ]:
catalog.list()

## Process model data

```{txt}
Coordinate System:  HGI
Variables:
  Date_Time: date and time in ISO format [UT]
  hour: elapsed time since trajectory start [hr]
  r: radial coordinate in HGI [AU]
  phi: longitude coordinate in HGI [deg]
  Rho: density [amu/cm^3]
  Ux, Uy, Uz: bulk velocity components in HGI [km/s]
  Bx, By, Bz: magnetic field components in HGI [nT]
  Ti: ion temperature [K]
```

In [ ]:
#| export
bcols_hgi = ['bx', 'by', 'bz']
bcols_rtn = ['br', 'bt', 'bn']
vcols_hgi = ['ux', 'uy', 'uz']
vcols_rtn = ['ur', 'ut', 'un']


def hgi2rtn(df: pl.LazyFrame | pl.DataFrame):
    """Transform coordinates from HGI to RTN"""

    # Compute RTN components for B
    br = pl.col("bx") * pl.col("phi_rad").cos() + pl.col("by") * pl.col("phi_rad").sin()
    bt = -pl.col("bx") * pl.col("phi_rad").sin() + pl.col("by") * pl.col("phi_rad").cos()
    bn = pl.col("bz")
    
    # Compute RTN components for V
    ur = pl.col("ux") * pl.col("phi_rad").cos() + pl.col("uy") * pl.col("phi_rad").sin()
    ut = -pl.col("ux") * pl.col("phi_rad").sin() + pl.col("uy") * pl.col("phi_rad").cos()
    un = pl.col("uz")

    result = df.with_columns(
        phi_rad=pl.col("phi").radians(),
    ).with_columns(
        br=br,
        bt=bt,
        bn=bn,
        ur=ur,
        ut=ut,
        un=un
    ).drop(["phi", "phi_rad"] + bcols_hgi + vcols_hgi)
    return result


def process_model(df: pandas.DataFrame) -> pl.DataFrame:
    """Process the model data"""
    df_pl = pl.from_pandas(df).lazy()

    return (
        df_pl
        .with_columns(
            time=pl.col("Date_Time").str.to_datetime(time_unit="ns"),
        )
        .drop(["Date_Time", "hour"])
        .sort("time")
        .collect()
    )

In [ ]:
# | code-summary: load jno model data
raw_jno_mswim2d = catalog.load("model.raw_jno_mswim2d")
processed_jno_mswim2d = process_model(raw_jno_mswim2d)
jno_mswim2d_rtn = processed_jno_mswim2d.pipe(hgi2rtn)

In [ ]:
def overview(df: pl.DataFrame):
    """Overview of the data"""
    df_pd = df.to_pandas()
    df_pd.hvplot(x="time", y=bcols_rtn)
    
    b_fig = df_pd.hvplot.line(x="time", y=bcols_rtn)
    v_fig = df_pd.hvplot.line(x="time", y=vcols_rtn)
    rho_fig = df_pd.hvplot.line(x="time", y="rho", logy=True)
    Ti_fig = df_pd.hvplot.line(x="time", y="Ti", logy=True)
    return (b_fig + v_fig + rho_fig + Ti_fig).cols(1).opts(shared_axes=False)


jno_mswim2d_rtn.pipe(overview)

#### Pipelines

```{sh}
kedro run --namespace=model
```

In [ ]:
# | export
def create_pipeline(**kwargs) -> Pipeline:
    nodes = [
        node(process_model, inputs="raw_jno_mswim2d", outputs="processed_jno_mswim2d"),
        node(hgi2rtn, inputs="processed_jno_mswim2d", outputs="jno_mswim2d_rtn"),
    ]
    return pipeline(nodes, namespace="model")

## Compare JUNO data with model

We are using juno 1min data to compare with model data

In [1]:
from ids_finder.pipelines.juno.pipeline import download_juno_data, preprocess_jno

In [ ]:
node(
    download_juno_data,
    inputs=None,
    outputs="raw_jno_ss_se_1min",
    name="download_JUNO_data_1min",
),
node(
    preprocess_jno,
    inputs="raw_jno_ss_se_1min",
    outputs="preprocessed_jno_ss_se_1min",
    name="preprocess_JUNO_node_1min",
)

In [ ]:
#| code-summary: load jno data and resample to 1h to match model resolution
preprocessed_jno_ss_se_1min: pl.DataFrame = catalog.load('preprocessed_jno_ss_se_1min')
jno_ss_se_1min = preprocessed_jno_ss_se_1min.lazy().rename(
    {"BX SE": "br", "BY SE": "bt", "BZ SE": "bn"}
)

jno_ss_se_1h = jno_ss_se_1min.pipe(resample, every="1h", period='2h')

In [ ]:
import plotly.graph_objects as go;
from plotly_resampler import register_plotly_resampler
from plotly_resampler import FigureResampler
import plotly.express as px

In [ ]:
jno_mswim2d_1h = processed_jno_mswim2d.lazy()

In [ ]:
def _tf(df: pl.DataFrame):
    "temporal function to select interesting columns and add norm"
    cols = ["time", "br", "bt", "bn"]

    return df.select(cols).with_columns(
        b=pl_norm(bcols_rtn),
    )


jno_joint_1h_wide: pl.DataFrame = (
    jno_ss_se_1h.pipe(_tf).join(
        jno_mswim2d_1h.pipe(_tf),
        on="time",
        suffix="_model",
    )
).collect()

jno_joint_1h_long = pl.concat(
    [
        jno_ss_se_1h.pipe(_tf).with_columns(type=pl.lit("1h")),
        jno_mswim2d_1h.pipe(_tf).with_columns(type=pl.lit("1h_model")),
    ]
).collect()

#### Data Porfiling

Results are showed in the following links

[Timeseries Report Result](jno_model_ts.html)

[Comparison Report Result](jno_model_comparison.html)

In [ ]:
#| eval: false
from ydata_profiling import ProfileReport, compare

In [ ]:
from fastcore.utils import threaded

In [ ]:
@threaded
def get_report_t(df: pl.DataFrame, output, **kwargs):
    '''get report and save to file in a thread
    '''
    get_report(df, **kwargs).to_file(output)
    return output

def get_report(df: pl.DataFrame, **kwargs):
    return ProfileReport(
        df.to_pandas().set_index("time"), **kwargs
    )

def get_comparison_report(df: pl.DataFrame, compare_col=None, tsmode=False, **kwargs):
    
    dfs_dict: Dict[str, pl.DataFrame] = df.partition_by(compare_col, as_dict=True)
    
    if tsmode:
        raise NotImplementedError("tsmode for comparison is not implemented yet in `ydata_profiling`")
        # UnionMatchError: can not match type "list" to any type of "time_index_analysis.period" union: typing.Union[float,  
        
        # Notes: for `tsmode`, we need to match the time first
        # select common timestamps
        from functools import reduce
        basetimestamps = reduce(np.intersect1d, [df.get_column('time') for df in dfs_dict.values()])
        dfs_dict = {
            k: df.filter(pl.col("time").is_in(basetimestamps))
            for k, df in dfs_dict.items()
        }

        for k, df in dfs_dict.items():
            logger.info(f"{k}: {len(df)}")
    
    comparison_report = compare(
        [get_report(df, title=k, **kwargs) for k, df in dfs_dict.items()]
    )
    
    # Obtain merged statistics
    comparison_report.get_description()

    return comparison_report

In [ ]:
get_report_t(
    jno_joint_1h_wide,
    output="jno_model_ts.html",
    tsmode=True,
    title="JUNO Model Timeseries Report",
)

In [ ]:
get_comparison_report(jno_joint_1h_long, compare_col="type").to_file(
    "jno_model_comparison.html"
)

## Validation

#### Connect `python` with `R` kernel

In [ ]:
%load_ext rpy2.ipython

from ids_finder.utils.r import py2rpy_polars
conv_pl = py2rpy_polars()

In [ ]:
%%R
library(ggplot2)
library(ggpubr)
library(viridis)

library(glue)
library(arrow)

In [ ]:
import warnings

# This will filter out all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#| column: screen
fig = px.line(
    jno_joint_1h_long.sort("time"),
    x="time",
    y="b",
    color="type",
)
fig

### Compare directly with scatter plot

In [ ]:
def bb_jointplot(data):
    g = sns.jointplot(
        x = 'b',
        y = 'b_model',
        data = data,
        kind = 'hist',
    )
    return g

In [ ]:
g = bb_jointplot(jno_joint_1h_wide)
g.ax_marg_x.set_xlim(0, 5)
g.ax_marg_y.set_ylim(0, 5)

In [ ]:
%%R -i jno_joint_1h_wide -c conv_pl
p1 <- ggplot(jno_joint_1h_wide, aes(x=b, y=b_model) ) +
  geom_bin2d() +
  geom_density_2d( colour="white" ) +
  scale_fill_continuous(trans="log", type = "viridis") +
  stat_regline_equation() + 
  xlim(-0.1, 10) +  # Set x-axis limits
  ylim(-0.1, 10) +  # Set y-axis limits
  theme_pubr(legend = 'right')
  # theme(legend.position = c(0.8,0.8))
  # stat_density_2d(aes(fill = ..level..), geom = "polygon", colour="white")

p1

#### Test: remove outliers

In [ ]:
from pyod.models.ecod import ECOD

data = jno_joint_1h_wide[['b', 'b_model']]

clf = ECOD()
clf.fit(data)

In [ ]:
y_train_scores = clf.decision_scores_  # raw outlier scores on the train data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)

In [ ]:
bb_jointplot(jno_joint_1h_wide.filter(y_train_pred==1))
data = jno_joint_1h_wide.filter(y_train_pred==0)
g = sns.jointplot(
    x = 'b',
    y = 'b_model',
    data = data,
    kind = 'hist',
)

g.plot_joint(sns.kdeplot, color="r", zorder=0, levels=6)


In [ ]:
from kedro.pipeline import Pipeline, pipeline, node

def create_pipeline(**kwargs) -> Pipeline:
    return pipeline([
        node(process_model, inputs="jno_mswim2d", outputs="processed_jno_mswim2d", name="preprocess model data")
    ])